In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import itertools
import time
import heapq
from tqdm import tqdm

In [2]:
#from greedy_algo_for_searching import*

In [3]:
import numpy as np
def calculate_distance(point1, point2):
    #return np.linalg.norm(point2 - point1)
    #return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)
    # Convert RA and Dec from degrees to radians
    ra1,dec1 = point1[0], point1[1]
    ra2, dec2 = point2[0], point2[1]
    ra1 = np.radians(ra1)
    dec1 = np.radians(dec1)
    ra2 = np.radians(ra2)
    dec2 = np.radians(dec2)

    # Haversine formula
    dlon = ra2 - ra1
    dlat = dec2 - dec1
    a = np.sin(dlat/2)**2 + np.cos(dec1) * np.cos(dec2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # Calculate the angular distance (in radians)
    distance = c

    # Convert the angular distance from radians to degrees
    distance_deg = np.degrees(distance)

    return distance_deg

In [4]:
def normalize_probabilities(likelihoods):
    total_likelihood = np.sum(likelihoods)
    normalized_likelihoods = likelihoods / total_likelihood
    return normalized_likelihoods

In [5]:
def convert_distance_to_time(cutoff_distance, rate_of_movement): #2000 per minute
    if(cutoff_distance < 0):
        return cutoff_distance
    cutoff_time = cutoff_distance / rate_of_movement
    return cutoff_time
def convert_time_to_distance(cutoff_time, rate_of_movement): #2000 per minute
    cutoff_distance = cutoff_time * rate_of_movement
    return cutoff_distance



In [ ]:
def rate_to_area(rate_mpm):
    rate_mps = rate_mpm/60
    time_seconds = 1000 / rate_mps
    degrees_covered = 0.1 * time_seconds
    area = degrees_covered ** 2
    return area

def area_to_rate(area):
    degrees_covered = area ** 0.5
    time_seconds = degrees_covered / 0.1
    rate_mps = 1000 / time_seconds
    rate_mpm = rate_mps*60
    return rate_mpm


O(N^2) ALGO

In [7]:
def calculate_metric(probability, distance, n):
    return probability / (distance ** n)

def find_next_point_2(points, probabilities, current_point, path, n):
    next_point_candidates = []
    
    for i,(point, probability) in enumerate(zip(points, probabilities)):

        if tuple(point) not in path:
            distance = calculate_distance(np.array(point),np.array(current_point))
            metric = calculate_metric(probability, distance, n)
            next_point_candidates.append((i, metric))
    
    next_point_candidates.sort(key=lambda x: x[1], reverse=True)
    
    if next_point_candidates:
        next_point = next_point_candidates[0][0]
    else:
        next_point = None
    
    return next_point

def find_path_2(points, probabilities, n):
    sorted_points_probs = sorted(zip(points, probabilities), key=lambda x: x[1], reverse=True)
    sorted_points, sorted_probs = zip(*sorted_points_probs)
    
    start_point = sorted_points[0]
    start_point_index = np.where(np.array(points) == start_point)[0][0]
    index_path = [start_point_index]
    current_point = start_point
    point_path = [start_point]
    point_path_set = {tuple(start_point)}
    while len(index_path) < len(points):
        next_point = find_next_point_2(points, probabilities, current_point, point_path_set, n)
        
        if next_point is None:
            break
        
        index_path.append(next_point)
        point_path.append(points[next_point])
        point_path_set.add(tuple(points[next_point]))
        current_point = points[next_point]
    
    return index_path,point_path

In [8]:
def generate_paths_compounded2(points_arrays, likelihoods_arrays,n_s):
    #n_s = np.linspace(n_min, n_max, num_algos)
    
    paths_compounded = []  # Initialize value2 as an empty list
    index = range(len(points_arrays))
    n_array = []
    for i in index:
        execution_times = []
        array_values = []  # Initialize array_values for the current index
        n_values = []
        for n in n_s:
            start_time = time.time()
            points = points_arrays[i]
            likelihoods = likelihoods_arrays[i]
            #print("\n n =", n, "\n")
            n2 = n
            n_values.append(n2)
            v2,point_path = find_path_2(points, likelihoods, n)
            array_values.append(v2)
            end_time = time.time()
            execution_time = end_time - start_time
            execution_times.append(execution_time)
        n_array.append(n_values)
        paths_compounded.append(array_values)  # Append array_values to value2  
        #print(execution_times)
    return paths_compounded, n_array

O(N) ALGORITHM

In [9]:
def find_next_point_1(sorted_points, sorted_probs, current_point, n):
    point_1 = sorted_points[0]
    prob_1 = sorted_probs[0]
    
    metric_1 = calculate_metric(prob_1, calculate_distance(np.array(point_1),np.array(current_point)), n)
    
    if len(sorted_points) > 1:
        point_2 = sorted_points[1]
        prob_2 = sorted_probs[1]
        
        metric_2 = calculate_metric(prob_2, calculate_distance(np.array(point_2), np.array(current_point)), n)
        
        if metric_1 >= metric_2:
            next_point = point_1
            sorted_points = sorted_points[1:]
            sorted_probs = sorted_probs[1:]
        else:
            next_point = point_2
            sorted_points = sorted_points[0:1] + sorted_points[2:]
            sorted_probs = sorted_probs[0:1] + sorted_probs[2:]
    else:
        next_point = point_1
        sorted_points = []
        sorted_probs = []
    
    return next_point, sorted_points, sorted_probs

def find_path_1(points, probabilities, n):
    sorted_points_probs = sorted(zip(points, probabilities), key=lambda x: x[1], reverse=True)
    sorted_points, sorted_probs = zip(*sorted_points_probs)
    points = np.array(points)
    probabilities = np.array(probabilities)
    start_point = sorted_points[0]
    sorted_points = sorted_points[1:]
    sorted_probs = sorted_probs[1:]
    index_path = [np.where(np.all(points == start_point, axis=1))[0][0]]
    point_path = [start_point]
    current_point = start_point
    
    while len(index_path) < len(points):
        next_point, sorted_points, sorted_probs = find_next_point_1(sorted_points, sorted_probs, current_point, n)
        
        if next_point is None:
            break
        
        next_point_index = np.where(np.all(points == next_point, axis=1))[0][0]
        
        index_path.append(next_point_index)
        point_path.append(next_point)
        current_point = next_point
    
    return index_path, point_path

In [10]:
def generate_paths_compounded1(points_arrays, likelihoods_arrays,n_s):
    #n_s = np.linspace(n_min, n_max, num_algos)
    
    paths_compounded = []  # Initialize value2 as an empty list
    index = range(len(points_arrays))
    n_array = []
    for i in index:
        execution_times = []
        array_values = []  # Initialize array_values for the current index
        n_values = []
        for n in n_s:
            start_time = time.time()
            points = points_arrays[i]
            likelihoods = likelihoods_arrays[i]
            #print("\n n =", n, "\n")
            n2 = n
            n_values.append(n2)
            v2,point_path = find_path_1(points, likelihoods, n)
            array_values.append(v2)
            end_time = time.time()
            execution_time = end_time - start_time
            execution_times.append(execution_time)
        n_array.append(n_values)
        paths_compounded.append(array_values)  # Append array_values to value2
        #print(execution_times)
    return paths_compounded, n_array

O(N!) ALGORITHM

In [11]:
def calculate_expected_value_1(points, path, likelihoods, n):
    expected_value = 0.0
    total_distance = 0.0
    for i in range(len(path)):
        if i == (len(path)-1):
            expected_value += likelihoods[path[0]]
        else:
            current_point = points[path[i]]
            next_point = points[path[i + 1]],
            distance = calculate_distance(current_point, next_point)
            total_distance = total_distance + distance
            likelihood = likelihoods[path[i + 1]]  # Likelihood of the next point
            expected_value += likelihood/(total_distance ** n)

    return expected_value
def generate_all_paths(points):
    paths = []
    total_points = len(points)
    path_length = total_points

    # Generate permutations of path indices
    permutations = itertools.permutations(range(total_points), path_length)

    # Convert permutations to paths
    for perm in permutations:
        path = [p for p in perm]
        paths.append(path)

    return paths

def generate_plots_1(points_arrays, likelihoods_arrays, n):
    for points, likelihoods in zip(points_arrays, likelihoods_arrays):
        num_points = len(points)

        # Generate all possible paths
        all_paths = generate_all_paths(range(num_points))

        max_expected_value = 0
        max_path = None

        # Calculate and compare the expected values for all paths
        for path in all_paths:
            expected_value = calculate_expected_value_1(points, path, likelihoods, n)
            #print("Path:", path)
            #print("Expected Value:", expected_value)
            #print("---")
            if expected_value > max_expected_value:
                max_expected_value = expected_value
                max_path = path

        # Print the maximum expected value and corresponding path
        #print("Maximum Expected Value:", max_expected_value)
        #print("Corresponding Path:", max_path)
        # Plot the path with arrows
        #plot_path_with_arrow(points, likelihoods, max_path)
        return max_path


In [12]:
def generate_paths_compounded(points_arrays, likelihoods_arrays,n_s):

    value2 = []  # Initialize value2 as an empty list
    index = range(len(points_arrays))
    n_array = []
    for i in index:
        execution_times = []
        array_values = []  # Initialize array_values for the current index
        n_values = []
        for n in n_s:
            start_time = time.time()
            points = [points_arrays[i]]
            likelihoods = [likelihoods_arrays[i]]
            #print("\n n =", n, "\n")
            n2 = n
            n_values.append(n2)
            v2 = generate_plots_1(points, likelihoods, n)
            array_values.append(v2)
            end_time = time.time()
            execution_time = end_time - start_time
            execution_times.append(execution_time)
        n_array.append(n_values)
        value2.append(array_values)  # Append array_values to value2
    #print(execution_times)
    return value2, n_array

In [13]:
import numpy as np

def generate_probability_distribution(x, y, mean_x, mean_y, std_dev):
    """
    Generates a Gaussian probability distribution as a function of x and y coordinates.
    """
    exponent = -((x - mean_x)**2 + (y - mean_y)**2) / (2 * std_dev**2)
    probability = np.exp(exponent) / (2 * np.pi * std_dev**2)
    return probability

def generate_points_array_likelihoods_array(num_galaxies_list, num_points_in_a_set, std_dev, grid_size=1000):
    """
    Generates multiple sets of realizations, each with varying number of galaxies,
    in a 10x10 grid and associates likelihood values using a Gaussian probability distribution.
    """
    points_arrays = []
    likelihoods_arrays = []
    for num_galaxies in num_galaxies_list:
        points_set = []
        likelihoods_set = []
        mean_x = np.random.uniform(0, grid_size)
        mean_y = np.random.uniform(0, grid_size)

        for _ in range(num_points_in_a_set):
            # print("Mean x = ", mean_x)
            # print("Mean y = ", mean_y)

            points = np.random.uniform(0, grid_size, size=(num_galaxies, 2))
            likelihoods = generate_probability_distribution(points[:, 0], points[:, 1], mean_x, mean_y, std_dev)
            
            likelihoods = normalize_probabilities([likelihoods])[0]
            points_set.append(points)
            likelihoods_set.append(likelihoods)
        
        points_arrays.append(points_set)
        likelihoods_arrays.append(likelihoods_set)
    return points_arrays, likelihoods_arrays

# num_galaxies_list = [3, 4, 5]
# num_points_in_a_set = 3

# points_arrays, likelihoods_arrays = generate_points_array_likelihoods_array(num_galaxies_list, num_points_in_a_set)

# print("points = ",points_arrays,"\nprobabilities = ",likelihoods_arrays,"\n")

In [14]:
# import numpy as np

# def generate_probability_distribution(stellar_mass, peak_mass=1e9, std_dev=1e8):
#     """
#     Generates a probability distribution based on stellar mass.
#     """
#     # You can use any function here that maps stellar mass to probability
#     # For example, a Gaussian centered at peak_mass
#     exponent = -((stellar_mass - peak_mass)**2) / (2 * std_dev**2)
#     probability = np.exp(exponent) / np.sqrt(2 * np.pi * std_dev**2)
#     return probability

# def generate_points_array_likelihoods_array(num_galaxies_list, num_points_in_a_set, peak_mass=1e9, std_dev=1e8, grid_size=1000):
#     """
#     Generates multiple sets of realizations, each with varying number of galaxies,
#     in a 10x10 grid and associates likelihood values based on stellar mass.
#     """
#     points_arrays = []
#     likelihoods_arrays = []
    
#     for num_galaxies in num_galaxies_list:
#         points_set = []
#         likelihoods_set = []

#         for _ in range(num_points_in_a_set):
#             # Generate random stellar masses (you can replace this with your actual data)
#             # will be replaced by actual data
#             stellar_masses = np.random.uniform(1e8, 1e10, size=num_galaxies)
            
#             # Calculate likelihoods based on stellar mass
#             likelihoods = generate_probability_distribution(stellar_masses, peak_mass, std_dev)
            
#             # Normalize the likelihoods
#             likelihoods = likelihoods / likelihoods.sum()
            
#             # Generate mock RA and Dec coordinates (x and y)
#             # Will be replaced by actual data
#             ra_dec_coordinates = np.random.uniform(0, grid_size, size=(num_galaxies, 2))
            
#             points_set.append(ra_dec_coordinates)
#             likelihoods_set.append(likelihoods)
        
#         points_arrays.append(points_set)
#         likelihoods_arrays.append(likelihoods_set)
    
#     return points_arrays, likelihoods_arrays, stellar_masses

# num_galaxies_list = [1, 1, 1]
# num_points_in_a_set = 2
# peak_mass = 1e9
# std_dev = 1e8
# grid_size = 1000

# points_arrays, likelihoods_arrays, stellar_masses = generate_points_array_likelihoods_array(num_galaxies_list, num_points_in_a_set, peak_mass, std_dev, grid_size)

# # Print or use the generated points and likelihoods as needed

# print("points = ",points_arrays,"\nprobabilities = ",likelihoods_arrays,"\nstellar mass = ",stellar_masses)